In [1]:
import pandas as pd
import numpy as np
import json
from emoji import core
import re

### 1. test 있는 컬럼만 남겨두기

In [81]:
test = pd.read_json('./res/test.json', typ='frame')

In [82]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10740 entries, 0 to 10739
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   tags          10740 non-null  object
 1   id            10740 non-null  int64 
 2   plylst_title  10740 non-null  object
 3   songs         10740 non-null  object
 4   like_cnt      10740 non-null  int64 
 5   updt_date     10740 non-null  object
dtypes: int64(2), object(4)
memory usage: 503.6+ KB


In [83]:
test.head()

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[],70107,,"[398985, 449403, 411543, 528044, 143048, 98020...",6,2012-09-29 01:57:26.000
1,"[나만의Best3, 인디아티스트들의추천음악]",7461,,"[196298, 269984, 267805, 175867, 529244, 63825...",0,2019-12-17 14:06:45.000
2,[드라이브],90348,,"[273433, 331003, 68432, 411659, 117793, 616860...",21,2015-05-23 10:44:48.000
3,[분위기],58617,,"[702227, 48152, 440008, 358488, 701041, 540721...",0,2019-03-14 09:47:34.000
4,[],102395,,"[630683, 481582, 528550, 285114, 506667, 17922...",38,2018-07-11 16:43:32.000


In [84]:
test_clear = test[test['plylst_title'] != ''].reset_index(drop=True)
test_clear

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[스트레스],88862,노래방 안에서,[],22,2016-01-22 20:55:12.000
1,[사랑],133975,봄바람과 듣고싶은 설레는 노래,[],0,2020-02-19 03:05:49.000
2,"[재즈, 기분전환, 뉴에이지]",135323,책읽으며 듣기좋은 잔잔 BGM♪,[],29,2017-06-13 09:46:27.000
3,[],43130,한여름의 센티멘탈 피아노.,[],4,2014-06-04 01:22:05.000
4,[새벽],113806,매력있고 편안한 노래들입니다!♬,[],2,2014-12-13 10:23:21.000
...,...,...,...,...,...,...
2036,[추억],27982,당신이 모르고 지나쳤을 수도 있는 명곡,[],11,2018-05-17 02:53:34.000
2037,[사랑],123364,달달하고 꽁냥스러운 염장송 (염장질에 천불주의),[],13,2016-11-08 10:33:32.000
2038,[기분전환],61817,피곤함을 달래줄 음악과 함께 하루의 피로를 풀자,[],0,2015-09-03 16:36:35.000
2039,[],59912,프랑스를 대표하는 일렉트로니카 아티스트,[],362,2010-01-08 14:53:06.000


In [86]:
test_clear = test_clear[test_clear['tags'].apply(len) > 0]
test_clear

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[스트레스],88862,노래방 안에서,[],22,2016-01-22 20:55:12.000
1,[사랑],133975,봄바람과 듣고싶은 설레는 노래,[],0,2020-02-19 03:05:49.000
2,"[재즈, 기분전환, 뉴에이지]",135323,책읽으며 듣기좋은 잔잔 BGM♪,[],29,2017-06-13 09:46:27.000
4,[새벽],113806,매력있고 편안한 노래들입니다!♬,[],2,2014-12-13 10:23:21.000
5,"[Pop, 산책, 인디]",126210,조아요 누르고 또 조아요 누를뻔한 들을때마다 좋은곡(상시업뎃),[],1,2019-04-28 20:21:38.000
...,...,...,...,...,...,...
2034,"[봄캐롤, 긱스, 버스커버스커, 봄]",113125,봄을 맞이하는 우리들의 자세,[],16,2019-03-13 15:20:00.000
2036,[추억],27982,당신이 모르고 지나쳤을 수도 있는 명곡,[],11,2018-05-17 02:53:34.000
2037,[사랑],123364,달달하고 꽁냥스러운 염장송 (염장질에 천불주의),[],13,2016-11-08 10:33:32.000
2038,[기분전환],61817,피곤함을 달래줄 음악과 함께 하루의 피로를 풀자,[],0,2015-09-03 16:36:35.000


In [91]:
test_clear = test_clear[test_clear['songs'].apply(len) > 0]
test_clear

,tags,id,plylst_title,songs,like_cnt,updt_date


In [92]:
test_clear['songs'].astype(str).nunique()

0

### 2. val 있는 컬럼만 남기기

In [93]:
val = pd.read_json('./res/val.json', typ='frame')

In [94]:
val.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23015 entries, 0 to 23014
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   tags          23015 non-null  object
 1   id            23015 non-null  int64 
 2   plylst_title  23015 non-null  object
 3   songs         23015 non-null  object
 4   like_cnt      23015 non-null  int64 
 5   updt_date     23015 non-null  object
dtypes: int64(2), object(4)
memory usage: 1.1+ MB


In [95]:
val.head()

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[],118598,,"[373313, 151080, 275346, 696876, 165237, 52593...",1675,2019-05-27 14:14:33.000
1,[],131447,앨리스테이블,[],1,2014-07-16 15:24:24.000
2,[],51464,,"[529437, 516103, 360067, 705713, 226062, 37089...",62,2008-06-21 23:26:22.000
3,[],45144,,"[589668, 21711, 570151, 320043, 13930, 599327,...",20,2017-10-30 18:15:43.000
4,[],79929,,"[672718, 121924, 102694, 683657, 201558, 38511...",20,2017-02-07 11:40:42.000


In [96]:
val_clear = val[val['plylst_title'] != '']
val_clear

,tags,id,plylst_title,songs,like_cnt,updt_date
1,[],131447,앨리스테이블,[],1,2014-07-16 15:24:24.000
8,[스트레스],80810,리듬타면서 빡시게 운동하자!!!(스트레스 날리자):},[],127,2017-02-09 17:33:45.000
9,[],142007,기분 좋은 재즈와 함께 만드는 달달한 하루,[],0,2015-06-22 09:11:02.000
17,[생각나],2380,다시 생각나는 그 사람,[],6,2016-03-19 23:53:20.000
20,"[피아노, 이루마, 메로디]",149069,불면증엔 아름다운 뉴에이지 곡들,[],11,2019-12-15 19:15:21.000
...,...,...,...,...,...,...
22991,[],32537,컨트리 황제 조니 캐시가 선 레코드 시절 발표한 초기 대표작,[],28,2019-06-17 14:22:48.000
22992,[사랑],32812,♥옷차림이 가벼워질때 부담없이 듣는음악♥,[],8,2015-05-08 00:09:01.000
22996,[],86721,해 저무는 밤,[],5,2016-04-27 15:32:55.000
23002,"[텐션, 명곡, 학창시절, 텐션업]",56570,숨어듣는명곡! 텐션올릴때듣는 그시절그노래!! (그냥명곡있을수있음주의),[],538,2020-04-18 22:02:07.000


In [98]:
val_clear['songs'].astype(str).unique()

array(['[]'], dtype=object)